In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

Intitializing Scala interpreter ...

Spark Web UI available at http://dd486c79897e:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1599031835620)
SparkSession available as 'spark'


import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 3.0.0


In [16]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)


inputStreamPath: String = /home/jovyan/data/events-stream
modelPath: String = /home/jovyan/models/spark-ml-model
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(tweet,StringType,true))
inputDF: org.apache.spark.sql.DataFrame = [tweet: string]


In [17]:
val sameModel = PipelineModel.load(modelPath)


val predictionDF = sameModel.transform(inputDF)

sameModel: org.apache.spark.ml.PipelineModel = pipeline_a2a460ee428f
predictionDF: org.apache.spark.sql.DataFrame = [tweet: string, words: array<string> ... 4 more fields]


In [29]:
val query = predictionDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
}.start()

query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3be7aac4


+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|@AlteracVolley wh...|[@alteracvolley, ...|(1000,[18,26,63,1...|[5.02952448837956...|[0.50295244883795...|       0.0|
|Just found I have...|[just, found, i, ...|(1000,[61,125,208...|[5.11777615296168...|[0.51177761529616...|       0.0|
|Aw. Scarhead is m...|[aw., scarhead, i...|(1000,[2,17,61,20...|[4.93060780565622...|[0.49306078056562...|       1.0|
|@brandelion still...|[@brandelion, sti...|(1000,[24,47,135,...|[4.86096528196380...|[0.48609652819638...|       1.0|
|@Votney_925 I hav...|[@votney_925, i, ...|(1000,[166,307,36...|[5.02952448837956...|[0.50295244883795...|       0.0|
|@NAKEDdmblauren f...|[@nakeddmblauren,...|(1000,[3,17,1

In [30]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

import org.apache.spark.sql.functions._
getProbability: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4815/0x0000000841799040@301ee573,DoubleType,List(Some(class[value[0]: vector])),None,false,true)


In [31]:
val query = predictionDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.select(getProbability($"probability").alias("clean_probability")).show
}.start()

query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@43bd5098


+-------------------+
|  clean_probability|
+-------------------+
|0.49704755116204347|
|0.48822238470383106|
| 0.5069392194343771|
| 0.5139034718036193|
|0.49704755116204347|
|0.49286308743091023|
| 0.5139034718036193|
| 0.4930069168489628|
| 0.5974890454714797|
| 0.5413017456772555|
| 0.5139034718036193|
|0.49704755116204347|
| 0.4909974038751022|
|0.47529570246457664|
+-------------------+

+-------------------+
|  clean_probability|
+-------------------+
| 0.4882079887649381|
|0.49782470864708317|
| 0.4958758733267194|
| 0.5140593149188744|
|0.47529570246457664|
| 0.4854641134327083|
| 0.4798735275587374|
| 0.5123212416919851|
+-------------------+

+-------------------+
|  clean_probability|
+-------------------+
|0.49704755116204347|
|0.45670492554831155|
|0.48822238470383106|
| 0.5168802578503527|
|0.39557637959940356|
|  0.526119118516318|
| 0.5139034718036193|
|0.49704755116204347|
| 0.4593830686061343|
|0.49704755116204347|
| 0.5261663507026854|
| 0.5046838017993076|
|0.41458